In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from norm import *
from utils import *
from MostJoints.getMostInformativeJoints import *
import os
from itertools import chain
import setting
import pickle

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
%matplotlib inline

In [2]:
Ns4_as2_list = np.array([[9,11,13,2],
[0,0,0,0],
[0,0,0,0],
[9,11,13,20],
[0,0,0,0],
[0,0,0,0],
[9,11,13,2],
[9,11,13,2],
[9,11,13,2],
[0,0,0,0],
[10,11,12,13],
[11,13,12,9],
[0,0,0,0],
[17,19,15,12],
[0,0,0,0],
[0,0,0,0],
[0,0,0,0],
[0,0,0,0],
[0,0,0,0],
[0,0,0,0]])

In [3]:
Ns4_as2_list = Ns4_as2_list-1

In [4]:
DATA_PATH = 'data/'
ACTION_SETS_PATH = 'actionsets/'
ACTION_SETS  = ['actionsets/ActionSet1.txt', 'actionsets/ActionSet2.txt', 'actionsets/ActionSet3.txt']
ACTION_NAMES = ['Wave','Hammer','Smash','Catch','Forward Punch','Throw','Draw X','Draw Tick','Draw ',
               'Circle','Clapping Hand','Two hand Wave','Side Boxing','Bend','Forward Kick','Side Kick','Jogging',
               'Tennis Swing','Tennis Serve','Golf Swing','Pickup&throw']
TRAINING_SUBJECTS = ['s01','s03','s05','s07','s09']
TESTING_SUBJECTS = ['s02','s04','s06','s08','s10']

with open(ACTION_SETS_PATH + 'ActionSet2.txt', 'r') as f:
    as3_names = f.readlines()
for i in range(len(as3_names)):
    as3_names[i] = as3_names[i].rstrip("\n")

In [5]:
train_as3_names = [as3_names[i] for i in range(len(as3_names)) if as3_names[i][4:7] in TRAINING_SUBJECTS]
test_as3_names = [as3_names[i] for i in range(len(as3_names)) if as3_names[i][4:7] in TESTING_SUBJECTS]
train_as3_names = [train_as3_names[i] + '_skeleton3D.txt' for i in range(len(train_as3_names))]
test_as3_names = [test_as3_names[i] + '_skeleton3D.txt' for i in range(len(test_as3_names))]
as3_names = [as3_names[i] + '_skeleton3D.txt' for i in range(len(as3_names))]
train_as3_names = sorted(train_as3_names)
test_as3_names = sorted(test_as3_names)
as3_names = sorted(as3_names)

In [6]:
def reshape_skeleton(X):
    noFrames = int(X.shape[0]/setting.NUMBER_OF_JOINTS)
    reshaped_X = []
    for i in range(setting.NUMBER_OF_JOINTS):
        reshaped_X.append([X[j] for j in range(i, X.shape[0], 20)])
    reshaped_X = np.asarray(reshaped_X)
    return reshaped_X
def get_MIJ_matrices(filename):
    
    action_label = int(filename[1:3])
    
    with open('data/' + filename, 'r') as f:
        skeleton_matrix = [line.rstrip('\n') for line in f]
    for j in range(len(skeleton_matrix)):
        skeleton_matrix[j] = [float(coord) for coord in skeleton_matrix[j].split(' ')]
    
    skeleton_matrix = np.asarray(skeleton_matrix)
    noFrames = int(skeleton_matrix.shape[0]/setting.NUMBER_OF_JOINTS)
    x = skeleton_matrix[:,0]
    y = skeleton_matrix[:,1]
    z = skeleton_matrix[:,2]
    
    x = reshape_skeleton(x)
    y = reshape_skeleton(y)
    z = reshape_skeleton(z)
    t = np.arange(1, noFrames+1).T
    
    x = x[Ns4_as2_list[action_label-1]]
    y = y[Ns4_as2_list[action_label-1]]
    z = z[Ns4_as2_list[action_label-1]]

    return x,y,z,t
def calculateCovarianceMat(X, Y, Z, T, nLevels, overlap = False, timeVar=True):
    nFrames = X.shape[0]
    nJoins = X.shape[1]
    assert Y.shape[0] == nFrames
    assert Z.shape[0] == nFrames
    assert T.shape[0] == nFrames
    assert Y.shape[1] == nJoins
    assert Z.shape[1] == nJoins

    # Normalize skeleton coordinators

    normX = normCord(X)
    normY = normCord(Y)
    normZ = normCord(Z)
    normT = normSeT(T)

    # Create a list of full covariance matrices
    fullCovMats = [[] for i in range(nLevels)]
    covMats = [[] for i in range(nLevels)]

    if timeVar:
        sizeMatrix = nJoins*3+1
    else:
        sizeMatrix = nJoins*3

    listIdxMatrix = getValueMatrix(sizeMatrix) # get half of covariance matrix indexes


    for l in range(1,nLevels+1):
        # Compute covariance matrixes for each level
        nofMats = 2**(l-1)
        sizeWindow = 1/nofMats
        stepWindow = sizeWindow
        if overlap:
            stepWindow = stepWindow/2
            nofMats = nofMats*2-1
        startFrameTimes = [stepWindow*i for i in range(nofMats)]
        fullCovMats[l-1] = [[] for i in range(nofMats)]
        covMats[l-1] = [[] for i in range(nofMats)]
        for i in range(len(startFrameTimes)):
            startTime = startFrameTimes[i]
            endTime = startFrameTimes[i] + sizeWindow + 2*np.finfo(float).eps
            sliceInds = [j for j in range(T.shape[0]) if normT[j] >= startTime and normT[j] < endTime]
            sliceX = normX[sliceInds, :]
            sliceY = normY[sliceInds, :]
            sliceZ = normZ[sliceInds, :]
            sliceT = normT[sliceInds]
            if not timeVar:
                sliceVars = np.concatenate((np.concatenate((sliceX,sliceY), axis=1), sliceZ), axis=1)
            else:
                sliceVars = np.concatenate((np.concatenate((sliceX, sliceY), axis=1), np.concatenate((sliceZ, sliceT), axis=1)), axis=1)
            covarianceMat = np.cov(sliceVars.T)
            fullCovMats[l-1][i] = covarianceMat
            # Get half of covarianceMat and save it as a vector (1-D matrix)
            one_half_vector = []
            mask = np.zeros_like(covarianceMat, dtype=np.bool)
            mask[np.triu_indices_from(mask)] = True
            for row in range(covarianceMat.shape[0]):
                for column in range(covarianceMat.shape[0]):
                    if mask[row][column] == True:
                        one_half_vector.append(covarianceMat[row][column])
            covMats[l-1][i] = np.asarray(one_half_vector)
        covMats[l-1] = np.asarray(covMats[l-1])
        
    covMats = np.asarray(covMats)
    vec = np.empty(0)
    for i in range(covMats.shape[0]):
        for j in range(covMats[i].shape[0]):
            vec = np.hstack((vec, covMats[i][j]))
    return fullCovMats, vec

def get_covariance_vector(x,y,z,t):
    fullCovmat, vec_covMat = calculateCovarianceMat(x.T, y.T, z.T, t, nLevels=3, overlap=True, timeVar=False)
    return vec_covMat

def make_preprocessed_training_data(filenames):
    X_train, y_train = [], []
    for i in range(len(filenames)):
        x, y, z, t = get_MIJ_matrices(filenames[i])
        vec_covMat = get_covariance_vector(x,y,z,t)
        label = int(filenames[i][1:3])
        X_train.append(vec_covMat)
        y_train.append(label)
    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    return X_train, y_train

In [7]:
X_train, y_train = make_preprocessed_training_data(train_as3_names)
model = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
model_name = 'as2_svm_model.pkl'
with open(model_name, 'wb') as f:
    pickle.dump(model, f)
X_test, y_test = make_preprocessed_training_data(test_as3_names)
y_hat = model.predict(X_test)

In [8]:
def get_acc(y_test, y_hat):
    count = 0
    for i in range(y_test.shape[0]):
        if y_test[i] == y_hat[i]:
            count += 1
    acc = (count/y_test.shape[0])*100
    return acc
print(get_acc(y_test, y_hat))

91.96428571428571


In [9]:
from sklearn.metrics import f1_score
f1_score(y_test, y_hat, average='micro')  

0.9196428571428571

In [18]:
from sklearn.svm import SVC
skeleton_path = 'skeleton-reality-data/re_skeleton_raw_1.txt'
model_SVC = OneVsRestClassifier(SVC(probability=True, random_state=0)).fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [29]:
with open(skeleton_path, 'r') as f:
    skeleton_list = f.readlines()

action_11 = skeleton_list[430:610]
for i in range(len(action_11)):
    action_11[i] = action_11[i].rstrip('\n')
for i in range(len(action_11)):
    action_11[i] = action_11[i].split(' ')[:-1]

for i in range(len(action_11)):
    action_11[i] = [float(coor) for coor in action_11[i]]
    action_11[i] = action_11[i][:60]

    
Ns4_11 = np.array([9,11,13,2])
Ns4_11 = Ns4_11-1

x,y,z = [], [], []
for i in range(len(action_11)):
    for j in range(len(action_11[i])):
        if j%3 == 0:
            x.append(action_11[i][j])
        elif j%3 == 1:
            y.append(action_11[i][j])
        elif j%3 == 2:
            z.append(action_11[i][j])
x = np.asarray(x)
y = np.asarray(y)
z = np.asarray(z)
X = reshape_skeleton(x)
Y = reshape_skeleton(y)
Z = reshape_skeleton(z)

X = X[Ns4_11]
Y = Y[Ns4_11]
Z = Z[Ns4_11]

def make_preprocessed_reality_data():
    T = np.arange(1, X.shape[1]+1).T
    vec_covMat = get_covariance_vector(X,Y,Z,T)
    return vec_covMat

vec_covMat = make_preprocessed_reality_data()
X_reality_test = []
X_reality_test.append(vec_covMat)
X_reality_test = np.asarray(X_reality_test)
y_pred_proba = model_SVC.predict_proba(X_reality_test)
y_pred = model_SVC.predict(X_reality_test)
print(y_pred)
print(min(y_pred_proba))

[12]
[0.00175423 0.08890005 0.01055037 0.01278852 0.02384877 0.00348018
 0.41311504 0.44556284]


In [22]:
y_SVC = model_SVC.predict(X_test)

In [23]:
y_SVC

array([ 4, 12,  4,  1,  8,  1,  1,  1,  1,  1,  1,  1,  4,  4,  4,  7,  8,
        4, 12,  4,  4,  4,  4,  4,  7,  7,  7,  8,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  9,  8,  8, 14, 12,  8,  9,  7,  7,  9,  9,  9,  9,  9,  1, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14])

In [24]:
y_hat

array([ 4, 14,  1,  1,  8,  1,  1,  1,  1,  1,  1,  1,  4,  4,  4,  7,  8,
        4,  4,  4,  4,  4,  4,  4,  7,  7,  7,  8,  7,  7,  1,  7,  7,  7,
        7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  9,  9,  9,  9,  9,  9,  9,  9,  7,  9,  9,  9,  9,  9,  9, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 14, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14])

In [26]:
print(get_acc(y_test, y_SVC))

85.71428571428571
